In [1]:
## mmrotate1.x기반으로 진행 
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.0.0rc1
3.1.0
11.1
GCC 7.3


In [2]:
## config LOAD
from mmengine.config import Config
cfg = Config.fromfile('custom_CONFIG_HJS_VER1.py')

In [3]:
from mmdet.apis import init_detector, inference_detector
import mmrotate

checkpoint_file = 'best_weight/epoch_1260.pth'
model = init_detector(cfg, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: best_weight/epoch_1260.pth


In [4]:
import json
import glob as _glob
import os
import random
import math
import numpy as np
import csv

def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

csv.register_dialect(
    'mydialect',
    delimiter = ',',
    quotechar = '"',
    doublequote = True,
    skipinitialspace = True,
    lineterminator = '\r\n',
    quoting = csv.QUOTE_MINIMAL)

def writecsv(csvname,contents):
    f = open(csvname, 'a', newline='')
    wr = csv.writer(f)
    wr.writerow(contents)
    f.close()

### 모델 output은 각도를 포함한 5개의 값으로 구성되어있기 때문에 이를 DOTA 형대의 데이터로 되돌려줌 
def convert_xywha_to_8coords(xywha, is_clockwise=True):
    x, y, w, h, a = xywha
    angle = a if is_clockwise else -a

    lt_x, lt_y = -w / 2, -h / 2
    rt_x, rt_y = w / 2, - h/ 2
    rb_x, rb_y = w / 2, h / 2
    lb_x, lb_y = - w / 2, h / 2

    lt_x_ = lt_x * math.cos(angle) - lt_y * math.sin(angle) + x
    lt_y_ = lt_x * math.sin(angle) + lt_y * math.cos(angle) + y
    rt_x_ = rt_x * math.cos(angle) - rt_y * math.sin(angle) + x
    rt_y_ = rt_x * math.sin(angle) + rt_y * math.cos(angle) + y
    lb_x_ = lb_x * math.cos(angle) - lb_y * math.sin(angle) + x
    lb_y_ = lb_x * math.sin(angle) + lb_y * math.cos(angle) + y
    rb_x_ = rb_x * math.cos(angle) - rb_y * math.sin(angle) + x
    rb_y_ = rb_x * math.sin(angle) + rb_y * math.cos(angle) + y

    return [lt_x_, lt_y_, rt_x_, rt_y_, rb_x_, rb_y_, lb_x_, lb_y_]

In [5]:
all_imgs = glob('container_dataset/valid_images','*.png')
all_imgs.sort()
all_imgs[:5]

['container_dataset/valid_images/OBJ00607_PS3_K3_NIA0086.png',
 'container_dataset/valid_images/OBJ01347_PS3_K3_NIA0092.png',
 'container_dataset/valid_images/OBJ01357_PS3_K3_NIA0092.png',
 'container_dataset/valid_images/OBJ01527_PS3_K3_NIA0095.png',
 'container_dataset/valid_images/OBJ01675_PS3_K3_NIA0102.png']

In [ ]:
from mmrotate.apis import inference_detector_by_patches
import pandas as pd
output_file_nm = 'result.csv'

writecsv(output_file_nm, ['File', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'X3', 'Y3', 'X4' ,'Y4'])

nn = 0
for img_nm in all_imgs:
    #### patch 단위로 inference 진행
    results=inference_detector_by_patches(model, img_nm, [1024], [512], [3, 4, 5], dict(type='nms_rotated', iou_threshold=0.1))
    
    File = img_nm.split('/')[-1][:8]
    for i in range(results.pred_instances.scores.shape[0]):
        ## Confidence Score가 0.1 이상인경우만 사용 
        ## 본 대회의 평가 메트릭은 AP(Average Precision)로, 모델의 출력 결과에서 Confidence Score를 통해 임계 처리를 할 때 임계값은 0.1만 가능합니다.
        if results.pred_instances.scores[i] >= 0.1:
            coords_dotaformat = convert_xywha_to_8coords(results.pred_instances.bboxes[i].cpu().numpy())
            coords_dotaformat = np.clip(coords_dotaformat,0,1024)
            writecsv(output_file_nm, [File]+[results.pred_instances.scores[i].cpu().numpy()]+list(coords_dotaformat))
    nn+=1
    if nn%10==0:
        print(nn,' END!')

